#Data Collection


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Dataset/TRAIN_SET.zip'


In [ ]:
!unzip '/content/drive/MyDrive/Dataset/TEST_SET.zip'

#Image Preprocessing

In [ ]:
#Import The ImageDataGenerator Library
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator (rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2, 
                                    horizontal_flip=True)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
#Loading our data and performing data agumentation
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(
'/content/TRAIN_SET',target_size=(64, 64), batch_size=5,color_mode='rgb',class_mode='sparse')

Found 2626 images belonging to 5 classes.


In [ ]:
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('/content/TEST_SET',target_size=(64, 64), batch_size=5,color_mode='rgb',class_mode='sparse')


Found 1055 images belonging to 5 classes.


In [ ]:
print(x_train.class_indices) #checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [ ]:

print(x_test.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [ ]:
from collections import Counter as c 
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

#Model Building

In [ ]:
# Importing Neccessary Libraries
import numpy as np #used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of Layers
from tensorflow.keras import layers #A Layer consists of a tensor-in tensor-out computation function
#Dense Layer is the regular deeply connected neural network Layer
from tensorflow.keras.layers import Dense, Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model=Sequential()

### Creating the model


In [ ]:
# Initializing the CNN
classifier = Sequential()
# First convolution Layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution Layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution Layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Flattening the Layers
classifier.add(Flatten())

In [ ]:
classifier.add(Dense (units=128, activation='relu')) 
classifier.add(Dense (units=5, activation='softmax')) # softmax for more than 2

In [ ]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)              

In [ ]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

##Fitting the model

In [ ]:
classifier.fit_generator( generator=x_train, steps_per_epoch = len(x_train), epochs=20, validation_data=x_test, validation_steps = len(x_test)) # No of images in test set

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


526/526 [==============================] - 15s 14ms/step - loss: 0.1821 - accuracy: 0.9360 - val_loss: 0.0239 - val_accuracy: 1.0000
Epoch 2/20
526/526 [==============================] - 7s 14ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0445 - val_accuracy: 0.9782
Epoch 3/20
526/526 [==============================] - 8s 15ms/step - loss: 0.0567 - accuracy: 0.9848 - val_loss: 0.0134 - val_accuracy: 0.9924
Epoch 4/20
526/526 [==============================] - 8s 15ms/step - loss: 3.1465e-04 - accuracy: 1.0000 - val_loss: 0.0102 - val_accuracy: 0.9981
Epoch 5/20
526/526 [==============================] - 7s 13ms/step - loss: 1.1439e-04 - accuracy: 1.0000 - val_loss: 0.0106 - val_accuracy: 0.9943
Epoch 6/20
526/526 [==============================] - 7s 14ms/step - loss: 7.3579e-05 - accuracy: 1.0000 - val_loss: 0.0095 - val_accuracy: 0.9943
Epoch 7/20
526/526 [==============================] - 7s 14ms/step - loss: 4.1322e-05 - accuracy: 1.0000 - val_loss: 0.0113 - val_accuracy: 

In [ ]:
classifier.save('nutrition.h5')



In [ ]:
! tar -zcvf nutrition-analyzer.tgz nutrition.h5

nutrition.h5


#Train Model On IBM

In [ ]:
#IBM Deployment
!pip install watson-machine-learning-client

In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
  "url":"https://eu-gb.ml.cloud.ibm.com",
  "apikey":"6GnaqWQTqFVcFvGeWFlW3wiZ9LALti2pc2kB1WBhOEDk"
}

client = APIClient(wml_credentials)
client

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [ ]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:eu-gb:a/dced4fbe290c4320a2c9e11d16e82847:fbdf9d5c-a53a-4882-bd8d-cdb7b1355676::',
      'guid': 'fbdf9d5c-a53a-4882-bd8d-cdb7b1355676',
      'name': 'Watson Machine Learning-80',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'AI_Deploy',
    'scope': {'bss_account_id': 'dced4fbe290c4320a2c9e11d16e82847'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '20c879cb-e65d-40cd-98bb-0fac434fea73',
      'bucket_region': 'eu-gb-standard',
      'credentials': {'admin': {'access_key_id': '34c946ea2d6d4b2da24cf82ecfa0d0ae',
        'api_key': 'QCysFwrBOqezcrbSQuiFqhiHsB0HCcvh6jzWbRSldLTS',
        'secret_access_key': '4690270aa71694ba3d156d9c478f1f4438406bc118735c48',
        'service_id': 'ServiceId-55a7ed3d-d568-4576-82c7-23cf3e18e8e6'},
       'editor': {'access_key_id': '99f62714eaa840b39019589430ebbaa9',
        'api_key

In [ ]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------  ------------------------
ID                                    NAME       CREATED
a79eec6f-d446-412d-876c-fd891a2d63a2  AI_Deploy  2022-11-08T18:12:27.758Z
------------------------------------  ---------  ------------------------


In [ ]:
space_uid = "a79eec6f-d446-412d-876c-fd891a2d63a2"
space_uid

'a79eec6f-d446-412d-876c-fd891a2d63a2'

In [ ]:
client.set.default_space(space_uid)

'SUCCESS'

In [ ]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [ ]:
software_space_id = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_id

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [ ]:
model_details = client.repository.store_model(model = 'nutrition-analyzer.tgz', meta_props={
    client.repository.ModelMetaNames.NAME:"NUTRITION ANALYZER Model", 
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_id
})

In [ ]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-08T19:17:29.295Z',
  'id': '91696d78-4e29-4bed-896e-5aab006fe825',
  'modified_at': '2022-11-08T19:17:32.722Z',
  'name': 'NUTRITION ANALYZER Model',
  'owner': 'IBMid-6640044D1T',
  'resource_key': 'ef97eafa-bb8c-4f98-a9f8-71a7ee63e3a4',
  'space_id': 'a79eec6f-d446-412d-876c-fd891a2d63a2'},
 'system': {'warnings': []}}

In [ ]:
model_id = client.repository.get_model_id(model_details)
model_id

'91696d78-4e29-4bed-896e-5aab006fe825'

In [ ]:
client.repository.download(model_id, 'NUTRITION_ANALYZER_model.tar.gb')

Successfully saved model content to file: 'NUTRITION_ANALYZER_model.tar.gb'


'/content/NUTRITION_ANALYZER_model.tar.gb'